In [ ]:
# used when in Kaggle
# !pip install git+https://github.com/keepsimpler/zero

In [2]:
from zero import *

/home/fengwf/anaconda3/envs/zero/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fengwf/anaconda3/envs/zero/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fengwf/anaconda3/envs/zero/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/fengwf/anaconda3/envs/zer

#### First get imagewoof data

In [ ]:
def get_data(size, woof, bs, workers=None):
    if   size<=128: path = URLs.IMAGEWOOF_160 if woof else URLs.IMAGENETTE_160
    elif size<=224: path = URLs.IMAGEWOOF_320 if woof else URLs.IMAGENETTE_320
    else          : path = URLs.IMAGEWOOF     if woof else URLs.IMAGENETTE
    path = untar_data(path)

    n_gpus = num_distrib() or 1
    if workers is None: workers = min(8, num_cpus()//n_gpus)

    return (ImageList.from_folder(path).split_by_folder(valid='val')
            .label_from_folder().transform(([flip_lr(p=0.5)], []), size=size)
            .databunch(bs=bs, num_workers=workers)
            .presize(size, scale=(0.35,1))
            .normalize(imagenet_stats))

In [ ]:
data = get_data(size=128, woof=True, bs=64)

#### Experimental Schedule
0. Fix operation on node: `xception`
1. Fix parameters about neural networks.
  * `num_stages` : [4, 5]
  * `num_nodes` : [32,48,64]
  * `num_channels` : [32,64]
2. Chose a random graph model, then give coefficient values, then generate DAGs.
3. Create neural network model based on the DAGs.
4. Training data and evaluate performance of the model.
5. Repeat 2-5, until all the random graph models, their coefficients, and generated DAGs are traveled.

In [13]:
# 0 : fix operation on node
Unit = xception
option = None # option when add the unique input and output nodes for DAG.

In [14]:
# 1 : fix parameters about neural networks
num_stages = 4  # number of stages
num_nodes = 32  # number of nodes per stage
num_channels = 32  # number of channels per node

##### 2 : Chose a random graph model - Niche model

In [15]:
# chose coefficient of the niche model
c = 0.05  # connectance
#c = 1 / math.sqrt(num_nodes) / 2
print(c)

0.05


In [16]:
# generate random DAGs using niche model, and add unique input and output nodes for them
species = [num_nodes] * num_stages
connectances = [c] * num_stages
Gs = [after_DAG(niche(s, c), option=option) for s,c in zip(species, connectances)]

In [21]:
ns = [num_channels] * (num_stages+1)
model = ComplexNet(Gs, ns, Stage=NetworkOP, Unit=Unit, efficient=True)
model = nn.DataParallel(model, device_ids=[0, 2]) # the only working schema for parallel on `node87`.

In [22]:
params = num_params(model)
params

tensor(221674)

In [ ]:
learn = Learner(data, model, metrics=accuracy).to_fp16()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot(suggestion=True) 

In [ ]:
lr = 0.  # learning rate is decided and estimated by `lr_find`.
epochs = 5
filename = 'niche_{}_{}_{}_{}_params{}'.format(num_stages, num_nodes, num_channels, c, params)

In [ ]:
for _ in range(3):
    Gs = [after_DAG(niche(s, c), option=option) for s,c in zip(species, connectances)]
    model = ComplexNet(Gs, ns, Stage=NetworkOP, Unit=Unit, efficient=True)
    model = nn.DataParallel(model, device_ids=[0, 2]) # the only working schema for parallel on `node87`.
    learn = Learner(data, model, metrics=accuracy, callback_fns=[partial(callbacks.CSVLogger, filename=filename, append=True)]).to_fp16()
    learn.fit_one_cycle(epochs, lr) 